# Data Wrangling

In [80]:
import json
import pandas as pd

## Loading and processing the dataset

In [82]:
def load_data(csv_path, nrows=None):
    """
    Load and preprocess the Google Analytics customer revenue dataset.
    
    Parameters:
        csv_path (str): Path to the CSV file.
        nrows (int, optional): Number of rows to load for testing purposes. Loads all rows if None.
        
    Returns:
        pd.DataFrame: Processed DataFrame with flattened JSON columns and formatted data.
    """
    
    # Define columns that contain JSON data
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    # Load the CSV file with JSON columns parsed and fullVisitorId as a string
    df = pd.read_csv(
        csv_path,
        converters={column: json.loads for column in JSON_COLUMNS},  # Convert JSON columns to dictionaries
        dtype={'fullVisitorId': 'str'},  # Ensure fullVisitorId is loaded as a string for uniqueness
        nrows=nrows  # Load specified number of rows if provided
    )
    
    # Flatten each JSON column and merge back into the main DataFrame
    for column in JSON_COLUMNS:
        # Convert JSON data in the column to a DataFrame
        column_as_df = pd.json_normalize(df[column])
        
        # Rename the columns to indicate their source (e.g., device_isMobile)
        column_as_df.columns = [f"{column}_{subcolumn}" for subcolumn in column_as_df.columns]
        
        # Drop the original JSON column and merge the new flattened columns
        df = df.drop(column, axis=1).join(column_as_df)
    
    # Drop columns that are either not useful or too complex for initial analysis
    df.drop(columns=['customDimensions', 'hits'], inplace=True)
    
    # Fill missing values for key numerical columns with 0
    # - totals_pageviews: Number of pages viewed in a session, can be 0 if no pages were viewed
    # - totals_transactionRevenue: Revenue generated, fill missing with 0 as no revenue
    df.fillna({
        'totals_pageviews': 0,
        'totals_transactionRevenue': 0
    }, inplace=True)
    
    # Convert specific columns to integers for consistency
    df['device_isMobile'] = df['device_isMobile'].astype(int)  # Convert isMobile to 1 or 0
    df['totals_transactionRevenue'] = df['totals_transactionRevenue'].astype(int)  # Revenue as integer
    
    # Ensure all object (string) columns are consistently treated as strings
    cat_cols = df.select_dtypes(include='object').columns
    df[cat_cols] = df[cat_cols].astype(str)
    
    return df


In [84]:
# Load a subset (100,000 rows) of the data for testing purposes
df = load_data('../data/raw/dataset.csv', nrows=1000000)

# Inspect the first few rows to confirm the processing
display(df.head())

,channelGrouping,date,fullVisitorId,socialEngagementType,visitId,visitNumber,visitStartTime,device_browser,device_browserVersion,device_browserSize,...,trafficSource_adwordsClickInfo.criteriaParameters,trafficSource_referralPath,trafficSource_isTrueDirect,trafficSource_adContent,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_adwordsClickInfo.gclId,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_campaignCode
0,Organic Search,20171016,3162355547410993243,Not Socially Engaged,1508198450,1,1508198450,Firefox,not available in demo dataset,not available in demo dataset,...,not available in demo dataset,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,Referral,20171016,8934116514970143966,Not Socially Engaged,1508176307,6,1508176307,Chrome,not available in demo dataset,not available in demo dataset,...,not available in demo dataset,/a/google.com/transportation/mtv-services/bike...,nan,nan,nan,nan,nan,nan,nan,nan
2,Direct,20171016,7992466427990357681,Not Socially Engaged,1508201613,1,1508201613,Chrome,not available in demo dataset,not available in demo dataset,...,not available in demo dataset,nan,True,nan,nan,nan,nan,nan,nan,nan
3,Organic Search,20171016,9075655783635761930,Not Socially Engaged,1508169851,1,1508169851,Chrome,not available in demo dataset,not available in demo dataset,...,not available in demo dataset,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,Organic Search,20171016,6960673291025684308,Not Socially Engaged,1508190552,1,1508190552,Chrome,not available in demo dataset,not available in demo dataset,...,not available in demo dataset,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [86]:
df.shape

(1000000, 58)

In [88]:
df.to_csv('../data/raw/readable_dataset.csv', index=False)

## Data Cleaning and Transformation
### Drop unnecessary columns

In [89]:
# List of columns to drop
drop_columns = [
    'socialEngagementType', 'device_browser', 'device_browserVersion', 'device_browserSize',
    'device_operatingSystem', 'device_operatingSystemVersion', 'device_mobileDeviceBranding',
    'device_mobileDeviceModel', 'device_mobileInputSelector', 'device_mobileDeviceInfo', 
    'device_mobileDeviceMarketingName', 'device_flashVersion', 'device_language', 
    'device_screenColors', 'device_screenResolution', 'geoNetwork_region', 'geoNetwork_metro', 
    'geoNetwork_city', 'geoNetwork_cityId', 'geoNetwork_networkDomain', 'geoNetwork_latitude', 
    'geoNetwork_longitude', 'geoNetwork_networkLocation', 'totals_visits', 'trafficSource_keyword',
    'trafficSource_adwordsClickInfo.criteriaParameters', 'trafficSource_referralPath', 
    'trafficSource_adContent', 'trafficSource_adwordsClickInfo.page', 
    'trafficSource_adwordsClickInfo.slot', 'trafficSource_adwordsClickInfo.gclId', 
    'trafficSource_adwordsClickInfo.adNetworkType', 'trafficSource_adwordsClickInfo.isVideoAd', 
    'trafficSource_campaignCode', 'visitId', 'device_isMobile', 'geoNetwork_subContinent', 
    'totals_newVisits', 'totals_totalTransactionRevenue', 'totals_sessionQualityDim'
]

# Drop the columns
df = df.drop(columns=drop_columns)

# Display the remaining columns
display(df.head())

,channelGrouping,date,fullVisitorId,visitNumber,visitStartTime,device_deviceCategory,geoNetwork_continent,geoNetwork_country,totals_hits,totals_pageviews,totals_bounces,totals_timeOnSite,totals_transactions,totals_transactionRevenue,trafficSource_campaign,trafficSource_source,trafficSource_medium,trafficSource_isTrueDirect
0,Organic Search,20171016,3162355547410993243,1,1508198450,desktop,Europe,Germany,1,1,1,nan,nan,0,(not set),google,organic,nan
1,Referral,20171016,8934116514970143966,6,1508176307,desktop,Americas,United States,2,2,nan,28,nan,0,(not set),sites.google.com,referral,nan
2,Direct,20171016,7992466427990357681,1,1508201613,mobile,Americas,United States,2,2,nan,38,nan,0,(not set),(direct),(none),True
3,Organic Search,20171016,9075655783635761930,1,1508169851,desktop,Asia,Turkey,2,2,nan,1,nan,0,(not set),google,organic,nan
4,Organic Search,20171016,6960673291025684308,1,1508190552,desktop,Americas,Mexico,2,2,nan,52,nan,0,(not set),google,organic,nan


### Convert `visitStartTime` and `date` to Readable Format

In [92]:
# Convert 'visitStartTime' from Unix timestamp to datetime format
df['visitStartTime'] = pd.to_datetime(df['visitStartTime'], unit='s')


# Convert 'date' from YYYYMMDD format to datetime format
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')

### Using `date` for Feature Engineering

In [94]:
# Extract additional time-based features
df['day_of_week'] = df['date'].dt.dayofweek  # Monday=0, Sunday=6
df['month'] = df['date'].dt.month  # Month of the year

# Define the new column order by placing 'day_of_week' and 'month' right after 'date_converted'
column_order = ['date', 'day_of_week', 'month'] + [col for col in df.columns if col not in ['date', 'day_of_week', 'month']]

# Reindex the DataFrame to reflect the new column order
df = df[column_order]

display(df.head())

,date,day_of_week,month,channelGrouping,fullVisitorId,visitNumber,visitStartTime,device_deviceCategory,geoNetwork_continent,geoNetwork_country,totals_hits,totals_pageviews,totals_bounces,totals_timeOnSite,totals_transactions,totals_transactionRevenue,trafficSource_campaign,trafficSource_source,trafficSource_medium,trafficSource_isTrueDirect
0,2017-10-16,0,10,Organic Search,3162355547410993243,1,2017-10-17 00:00:50,desktop,Europe,Germany,1,1,1,nan,nan,0,(not set),google,organic,nan
1,2017-10-16,0,10,Referral,8934116514970143966,6,2017-10-16 17:51:47,desktop,Americas,United States,2,2,nan,28,nan,0,(not set),sites.google.com,referral,nan
2,2017-10-16,0,10,Direct,7992466427990357681,1,2017-10-17 00:53:33,mobile,Americas,United States,2,2,nan,38,nan,0,(not set),(direct),(none),True
3,2017-10-16,0,10,Organic Search,9075655783635761930,1,2017-10-16 16:04:11,desktop,Asia,Turkey,2,2,nan,1,nan,0,(not set),google,organic,nan
4,2017-10-16,0,10,Organic Search,6960673291025684308,1,2017-10-16 21:49:12,desktop,Americas,Mexico,2,2,nan,52,nan,0,(not set),google,organic,nan


### Check for duplicate rows

In [96]:
duplicate_rows = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_rows}")

Number of duplicate rows: 0


### Check for null values

In [98]:
df.isna().sum()

date                          0
day_of_week                   0
month                         0
channelGrouping               0
fullVisitorId                 0
visitNumber                   0
visitStartTime                0
device_deviceCategory         0
geoNetwork_continent          0
geoNetwork_country            0
totals_hits                   0
totals_pageviews              0
totals_bounces                0
totals_timeOnSite             0
totals_transactions           0
totals_transactionRevenue     0
trafficSource_campaign        0
trafficSource_source          0
trafficSource_medium          0
trafficSource_isTrueDirect    0
dtype: int64

### Replace 'nan' with 0 in Specific Columns

In [100]:
# Specify the columns to replace 'nan' with 0
columns_to_replace_nan = ['totals_bounces', 'totals_timeOnSite', 'totals_transactions']

# Replace 'nan' (string) with 0 in the specified columns
df[columns_to_replace_nan] = df[columns_to_replace_nan].replace('nan', 0)

# Replace 'nan' (string) with 'False' in trafficSource_isTrueDirect
df['trafficSource_isTrueDirect'] = df['trafficSource_isTrueDirect'].replace('nan', 'False')


display(df.head())

,date,day_of_week,month,channelGrouping,fullVisitorId,visitNumber,visitStartTime,device_deviceCategory,geoNetwork_continent,geoNetwork_country,totals_hits,totals_pageviews,totals_bounces,totals_timeOnSite,totals_transactions,totals_transactionRevenue,trafficSource_campaign,trafficSource_source,trafficSource_medium,trafficSource_isTrueDirect
0,2017-10-16,0,10,Organic Search,3162355547410993243,1,2017-10-17 00:00:50,desktop,Europe,Germany,1,1,1,0,0,0,(not set),google,organic,False
1,2017-10-16,0,10,Referral,8934116514970143966,6,2017-10-16 17:51:47,desktop,Americas,United States,2,2,0,28,0,0,(not set),sites.google.com,referral,False
2,2017-10-16,0,10,Direct,7992466427990357681,1,2017-10-17 00:53:33,mobile,Americas,United States,2,2,0,38,0,0,(not set),(direct),(none),True
3,2017-10-16,0,10,Organic Search,9075655783635761930,1,2017-10-16 16:04:11,desktop,Asia,Turkey,2,2,0,1,0,0,(not set),google,organic,False
4,2017-10-16,0,10,Organic Search,6960673291025684308,1,2017-10-16 21:49:12,desktop,Americas,Mexico,2,2,0,52,0,0,(not set),google,organic,False


In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 20 columns):
 #   Column                      Non-Null Count    Dtype         
---  ------                      --------------    -----         
 0   date                        1000000 non-null  datetime64[ns]
 1   day_of_week                 1000000 non-null  int32         
 2   month                       1000000 non-null  int32         
 3   channelGrouping             1000000 non-null  object        
 4   fullVisitorId               1000000 non-null  object        
 5   visitNumber                 1000000 non-null  int64         
 6   visitStartTime              1000000 non-null  datetime64[ns]
 7   device_deviceCategory       1000000 non-null  object        
 8   geoNetwork_continent        1000000 non-null  object        
 9   geoNetwork_country          1000000 non-null  object        
 10  totals_hits                 1000000 non-null  object        
 11  totals_pageviews         

### Convert columns to the appropriate types

In [104]:
# df['channelGrouping'] = df['channelGrouping'].astype(str)
# df['fullVisitorId'] = df['fullVisitorId'].astype(str)
# df['device_deviceCategory'] = df['device_deviceCategory'].astype(str)
# df['geoNetwork_continent'] = df['geoNetwork_continent'].astype(str)
# df['geoNetwork_country'] = df['geoNetwork_country'].astype(str)
# df['totals_hits'] = df['totals_hits'].astype(int)
# df['totals_pageviews'] = df['totals_pageviews'].astype(int)
# df['totals_bounces'] = df['totals_bounces'].astype(int)
# df['totals_sessionQualityDim'] = df['totals_sessionQualityDim'].astype(int)
# df['totals_timeOnSite'] = df['totals_timeOnSite'].astype(int)
# df['totals_transactions'] = df['totals_transactions'].astype(int)
# df['trafficSource_campaign'] = df['trafficSource_campaign'].astype(str)
# df['trafficSource_source'] = df['trafficSource_source'].astype(str)
# df['trafficSource_medium'] = df['trafficSource_medium'].astype(str)
df['trafficSource_isTrueDirect'] = df['trafficSource_isTrueDirect'].astype(bool)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 20 columns):
 #   Column                      Non-Null Count    Dtype         
---  ------                      --------------    -----         
 0   date                        1000000 non-null  datetime64[ns]
 1   day_of_week                 1000000 non-null  int32         
 2   month                       1000000 non-null  int32         
 3   channelGrouping             1000000 non-null  object        
 4   fullVisitorId               1000000 non-null  object        
 5   visitNumber                 1000000 non-null  int64         
 6   visitStartTime              1000000 non-null  datetime64[ns]
 7   device_deviceCategory       1000000 non-null  object        
 8   geoNetwork_continent        1000000 non-null  object        
 9   geoNetwork_country          1000000 non-null  object        
 10  totals_hits                 1000000 non-null  object        
 11  totals_pageviews         

In [106]:
df.shape

(1000000, 20)

In [108]:
display(df.head())

,date,day_of_week,month,channelGrouping,fullVisitorId,visitNumber,visitStartTime,device_deviceCategory,geoNetwork_continent,geoNetwork_country,totals_hits,totals_pageviews,totals_bounces,totals_timeOnSite,totals_transactions,totals_transactionRevenue,trafficSource_campaign,trafficSource_source,trafficSource_medium,trafficSource_isTrueDirect
0,2017-10-16,0,10,Organic Search,3162355547410993243,1,2017-10-17 00:00:50,desktop,Europe,Germany,1,1,1,0,0,0,(not set),google,organic,True
1,2017-10-16,0,10,Referral,8934116514970143966,6,2017-10-16 17:51:47,desktop,Americas,United States,2,2,0,28,0,0,(not set),sites.google.com,referral,True
2,2017-10-16,0,10,Direct,7992466427990357681,1,2017-10-17 00:53:33,mobile,Americas,United States,2,2,0,38,0,0,(not set),(direct),(none),True
3,2017-10-16,0,10,Organic Search,9075655783635761930,1,2017-10-16 16:04:11,desktop,Asia,Turkey,2,2,0,1,0,0,(not set),google,organic,True
4,2017-10-16,0,10,Organic Search,6960673291025684308,1,2017-10-16 21:49:12,desktop,Americas,Mexico,2,2,0,52,0,0,(not set),google,organic,True


In [110]:
df.to_csv('../data/clean/dataset.csv', index=False)